<a href="https://colab.research.google.com/github/Irohasu500/02Robocode/blob/main/CNN_analyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content")
#ファイルはすべてドライブ上で行う。

# 初めてやる場合は以下の処理が必要。
!unzip "./drive/MyDrive/American.zip";

Archive:  ./drive/MyDrive/American.zip
  inflating: American/0/hand1_0_bot_seg_1_cropped.jpeg  
  inflating: American/0/hand1_0_bot_seg_2_cropped.jpeg  
  inflating: American/0/hand1_0_bot_seg_3_cropped.jpeg  
  inflating: American/0/hand1_0_bot_seg_4_cropped.jpeg  
  inflating: American/0/hand1_0_bot_seg_5_cropped.jpeg  
  inflating: American/0/hand1_0_dif_seg_1_cropped.jpeg  
  inflating: American/0/hand1_0_dif_seg_2_cropped.jpeg  
  inflating: American/0/hand1_0_dif_seg_3_cropped.jpeg  
  inflating: American/0/hand1_0_dif_seg_4_cropped.jpeg  
  inflating: American/0/hand1_0_dif_seg_5_cropped.jpeg  
  inflating: American/0/hand1_0_left_seg_1_cropped.jpeg  
  inflating: American/0/hand1_0_left_seg_2_cropped.jpeg  
  inflating: American/0/hand1_0_left_seg_3_cropped.jpeg  
  inflating: American/0/hand1_0_left_seg_4_cropped.jpeg  
  inflating: American/0/hand1_0_left_seg_5_cropped.jpeg  
  inflating: American/0/hand1_0_right_seg_1_cropped.jpeg  
  inflating: American/0/hand1_0_right_seg_

In [3]:
# 上のセルを実行すると、なぜかcontent上にAmericanが解凍される。
os.chdir("/content")

In [4]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation, Conv2D, Flatten, Dense,Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD, Adadelta, Adagrad, Adam, Adamax, RMSprop, Nadam

from PIL import Image
import numpy as np
import glob
import matplotlib.pyplot as plt
import time
import os

# for VGG16
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

# for Xception
from tensorflow.keras.applications import Xception

In [5]:
# Google Driveを起動した上で以下のコードを実行する。
os.chdir("/content")
print(os.getcwd())


/content


In [11]:
######################################### 設定項目 ####################################
# shallow, normal, ....VGG16,Xception
model_name = "normal"
#nb_classes = 2                          # 正解の種類
nb_classes = 36                          # 正解の種類

dataset_folder_name = "American"



In [15]:
######################################### 画像を取ってくる ####################################
"""
    ファイル構成
    cnn -----   dog     ---~---各画像ファイル
          |
          |
          ---   cat    ---~---各画像ファイル
"""

folder = os.listdir(dataset_folder_name)
print(folder)
image_size = 64                                                 # ここで画像をどのサイズに直すか(あまり小さくするとなんの画像かわからなくなる。)
dense_size  = len(folder)

X = []                                                          # 画像を保存する変数
Y = []                                                          # 画像のインデックスを保存する。
for index, name in enumerate(folder):
    #print("index:" , index , "name" , name)
    dir = "./" + dataset_folder_name + "/" + name
    files = glob.glob(dir + "/*.jpeg")
    for i, file in enumerate(files):
        print(i , file);
        image = Image.open(file)
        #image.save("test_pre.png")
        image = image.convert("RGB")
        image = image.resize((image_size, image_size))
        #image.show()
        #image.save("test_after.png")
        data = np.asarray(image)
        X.append(data)
        Y.append(index)

# os.getcwd()
# os.getcwd()
# if np.all(X == 0):        # 別に全て0ではなかった
#     print('all zero!')
X = np.array(X)
Y = np.array(Y)
X = X.astype('float32')                                         # 型を変換。(npではこのようにする)
X = X / 255.0                                                    #正規化  
#print(X.shape)                                               # (1984, 64, 64, 3)

print(X)
print(Y)
Y = to_categorical(Y, dense_size)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.90)

['7', '9', '6', '8', '5', '2', '4', '1', '3', '0', 'j', 'a', 'f', 'c', 'e', 'b', 'h', 'i', 'g', 'd', 'r', 'p', 's', 'o', 't', 'k', 'q', 'm', 'l', 'n', 'y', 'u', 'w', 'x', 'z', 'v']
0 ./American/7/hand1_7_left_seg_4_cropped.jpeg
1 ./American/7/hand1_7_right_seg_2_cropped.jpeg
2 ./American/7/hand2_7_top_seg_4_cropped.jpeg
3 ./American/7/hand5_7_dif_seg_4_cropped.jpeg
4 ./American/7/hand2_7_bot_seg_4_cropped.jpeg
5 ./American/7/hand2_7_left_seg_5_cropped.jpeg
6 ./American/7/hand5_7_dif_seg_2_cropped.jpeg
7 ./American/7/hand1_7_top_seg_2_cropped.jpeg
8 ./American/7/hand2_7_dif_seg_1_cropped.jpeg
9 ./American/7/hand2_7_dif_seg_2_cropped.jpeg
10 ./American/7/hand4_7_bot_seg_4_cropped.jpeg
11 ./American/7/hand2_7_dif_seg_4_cropped.jpeg
12 ./American/7/hand2_7_dif_seg_5_cropped.jpeg
13 ./American/7/hand3_7_dif_seg_1_cropped.jpeg
14 ./American/7/hand1_7_dif_seg_4_cropped.jpeg
15 ./American/7/hand1_7_top_seg_5_cropped.jpeg
16 ./American/7/hand1_7_top_seg_1_cropped.jpeg
17 ./American/7/hand2_7_bo

In [ ]:
#########################  モデルを作成する #########################################################
def normal_CNN():
    """
        畳み込み層は4つ。(これはCNNでは少し深いため、勾配消失の可能性あり。)
        
        Dropoutを用いて、過学習を避けている？

        勾配消失を避けるため、reluがよく選択される。
    """
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(dense_size))
    model.add(Activation('softmax'))

    # モデルの説明を表示する。
    model.summary()

    return model

def secondary_CNN_shallow_ver():
    """
        畳み込み層は4つ。(これはCNNでは少し深いため、勾配消失の可能性あり。)
        
        Dropoutを用いて、過学習を避けている？

        勾配消失を避けるため、reluがよく選択される。
    """
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(dense_size))
    model.add(Activation('softmax'))

    # モデルの説明を表示する。
    model.summary()

    return model

def VGG16_CNN():
    # VGG-16モデルの構造と重みをロード
    # include_top=Falseによって、VGG16モデルから全結合層を削除
    input_tensor = Input(shape=(image_size, image_size,3))
    vgg16_model = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

    # 全結合層の構築
    top_model = Sequential()
    top_model.add(Flatten(input_shape=vgg16_model.output_shape[1:]))
    top_model.add(Dense(256))
    top_model.add(Activation("relu"))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(nb_classes))
    top_model.add(Activation("softmax"))

    # 全結合層を削除したVGG16モデルと上で自前で構築した全結合層を結合
    #model = Model(input=vgg16_model.input, output=top_model(vgg16_model.output)) # エラー
    model = Model(vgg16_model.input, top_model(vgg16_model.output))

    # 図3における14層目までのモデル重みを固定（VGG16のモデル重みを用いる）
    #for layer in model.layers[:15]:
    #        layer.trainable = False

    # モデルのコンパイル
    # model.compile(loss='categorical_crossentropy',
    #               optimizer=SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True),
    #               metrics=['accuracy'])
    model.summary()
    return model

def Xception_CNN():
    # VGG-16モデルの構造と重みをロード
    # include_top=Falseによって、VGG16モデルから全結合層を削除
    input_tensor = Input(shape=(image_size, image_size,3))
    Xception_model = Xception(include_top=False, weights='imagenet', input_tensor=input_tensor)

    # 全結合層の構築
    top_model = Sequential()
    top_model.add(Flatten(input_shape=Xception_model.output_shape[1:]))
    top_model.add(Dense(256))
    top_model.add(Activation("relu"))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(nb_classes))
    top_model.add(Activation("softmax"))

    # 全結合層を削除したVGG16モデルと上で自前で構築した全結合層を結合
    #model = Model(input=vgg16_model.input, output=top_model(vgg16_model.output)) # エラー
    model = Model(Xception_model.input, top_model(Xception_model.output))

    # 図3における14層目までのモデル重みを固定（VGG16のモデル重みを用いる）
    #for layer in model.layers[:15]:
    #        layer.trainable = False

    # モデルのコンパイル
    # model.compile(loss='categorical_crossentropy',
    #               optimizer=SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True),
    #               metrics=['accuracy'])
    model.summary()
    return model

"""
    モデルを選択して、返す関数。
"""
def Model_factory(model_algo_name):
    if model_algo_name == "normal":
        return normal_CNN()
    elif model_algo_name == "shallow":
        return secondary_CNN_shallow_ver()
    elif model_algo_name == "VGG16":
        return VGG16_CNN()
    elif model_algo_name == "Xception":
        return Xception_CNN()
    else:
        return -1
    pass

In [ ]:

######################################## モデルを使って学習する #################################################
model = Model_factory(model_name)

optimizers ="Adadelta"                      # 学習の最適化処理は
results = {}                                # 学習の結果(!モデルではない)を保存する変数
epochs = 250                                # エポック数の指定 (エポック数 = 1つの同じデータの塊を何回学習に使うか？)
model.compile(loss='categorical_crossentropy', optimizer=optimizers, metrics=['accuracy'])      # 誤差関数：交差エントロピー誤差  評価指標：正解率  アルゴリズム：
results= model.fit(X_train, y_train, validation_split=0.2, epochs=epochs )

os.chdir("drive/MyDrive")

model_json_str = model.to_json()                                # モデルをjsonにして保存する。
jsonfile_name = model_name + str(epochs) +  "_model.json"
open(jsonfile_name, 'w').write(model_json_str)         # ファイルを開いて書き込む
h5file_name = model_name + str(epochs) + "_weights.h5"
model.save_weights(h5file_name)                      # h5ファイルで保存。



x = range(epochs)
print(results)






def compare_TV(history):
        import matplotlib.pyplot as plt

        # Setting Parameters
        acc = history.history['accuracy']
        val_acc = history.history['val_accuracy']
        loss = history.history['loss']
        val_loss = history.history['val_loss']

        epochs = range(len(acc))

        # 1) Accracy Plt
        plt.plot(epochs, acc, 'bo' ,label = 'training acc')
        plt.plot(epochs, val_acc, 'b' , label= 'validation acc')
        plt.title('Training and Validation acc')
        plt.legend()

        plt.figure()

        # 2) Loss Plt
        plt.plot(epochs, loss, 'bo' ,label = 'training loss')
        plt.plot(epochs, val_loss, 'b' , label= 'validation loss')
        plt.title('Training and Validation loss')
        plt.legend()

        plt.show()
compare_TV(results)
exit(1)






# 以下のコードは間違っている、historyオブジェクトが使えてない。
for k, result in results.items():
    plt.plot(x, result.history['acc'], label=k)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5),borderaxespad=0, ncol=2)

name = 'acc.jpg'
plt.savefig(name, bbox_inches='tight')
plt.close()

for k, result in results.items():
    plt.plot(x, result.history['val_acc'], label=k)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5),borderaxespad=0, ncol=2)

name = 'val_acc.jpg'
plt.savefig(name, bbox_inches='tight')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['cat', 'dog']


KeyboardInterrupt: ignored